In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
seed = 18

merged_navigational_df = pd.read_csv("../../../../data/processed_for_model/merged_themes_using_jaccard_method/merged_Navigational_sentence_level_batch_1_jaccard.csv", encoding='utf-8')

# Shuffle the merged dataset
merged_navigational_df = shuffle(merged_navigational_df, random_state=seed)

# Train-test split 
training_df, test_df = train_test_split(merged_navigational_df, test_size=0.2, random_state=18, stratify=merged_navigational_df['label'])

training_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [2]:
print(f"Training dataset shape: {training_df.shape} \nTest dataset shape: {test_df.shape}")
pos_labels = len([n for n in training_df['label'] if n==1])
print("Positive labels present in the dataset : {}  out of {} or {}%".format(pos_labels, len(training_df['label']), (pos_labels/len(training_df['label']))*100))
pos_labels = len([n for n in test_df['label'] if n==1])
print("Positive labels present in the test dataset : {}  out of {} or {}%".format(pos_labels, len(test_df['label']), (pos_labels/len(test_df['label']))*100))

Training dataset shape: (2556, 3) 
Test dataset shape: (640, 3)
Positive labels present in the dataset : 227  out of 2556 or 8.881064162754305%
Positive labels present in the test dataset : 57  out of 640 or 8.90625%


In [3]:
training_df

,sentence,label,phrase
0,i also have to take physics 2 next semester so...,0,['I am in this class because I am not good at ...
1,after graduating high school there are three p...,0,"[""I'm here at this institution becauase in ord..."
2,these are valuable skills that will definitely...,0,['If I am going to be completely honest Im her...
3,there are certain aspects of physics that i fi...,0,['it is a requirement for me to take Physics 1...
4,"i want to represent myself, my gender, my sexu...",0,['Im currently lost as to how it is that I wan...
...,...,...,...
2551,this is one of them.,0,"['I am interested in the subject of Physics, i..."
2552,i have an obvious interest in studying how the...,1,"['On an academic note, I have to take this cla..."
2553,"as a business major, i needed to complete a la...",1,"['As a business major, I needed to complete a ..."
2554,"i am here because of my parents, my parents fl...",0,['My parents both didnt want their kids to liv...


In [4]:
MAXLEN = 150

In [5]:
X = training_df['sentence']
y = training_df['label']

In [6]:
import ktrain
from ktrain import text

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 18, stratify=y)

model_name = 'bert-base-cased'

distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
distillbert_base_model = distillbert_transformer.get_classifier()


Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

preprocessing train...
language: en
train sequence lengths:
	mean : 21
	95percentile : 39
	99percentile : 53


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 21
	95percentile : 41
	99percentile : 58


In [12]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Define classes and class labels
classes = np.array([0, 1])
class_labels = list(training_df.label)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=class_labels)

# Print class weights
print(class_weights)

class_weights = {0:0.54873336,1:5.62995595}

[0.54873336 5.62995595]


In [8]:
import tensorflow as tf
import numpy as np
import os
import random
def reset_random_seeds(seed=2):
    os.environ['PYTHONHASHSEED']=str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [13]:
# Build BERT model
# model = text.text_classifier('distilbert', train_data=(X_train, y_train), preproc=distillbert_transformer)
learner = ktrain.get_learner(distillbert_base_model, train_data=training_set, val_data=validation_set, batch_size=6)
# learner.fit_onecycle(2e-5, 4, class_weight=class_weights)
# learner.autofit(2.27E-06, early_stopping=4)
learner.autofit(2e-5, early_stopping=4, class_weight=class_weights)
# distillbert_learner.set_weight_decay(0.001)
# distillbert_learner.autofit(2.27E-06, early_stopping=4, class_weight=class_weights)

reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 2e-05...
Epoch 1/1024
299/299 [==============================] - 118s 382ms/step - loss: 0.1933 - accuracy: 0.9324 - val_loss: 0.4297 - val_accuracy: 0.8475
Epoch 2/1024
299/299 [==============================] - 114s 379ms/step - loss: 0.2368 - accuracy: 0.9016 - val_loss: 0.3827 - val_accuracy: 0.8305
Epoch 3/1024
299/299 [==============================] - 108s 357ms/step - loss: 0.1609 - accuracy: 0.9430 - val_loss: 0.3246 - val_accuracy: 0.8931
Epoch 4/1024
299/299 [==============================] - 106s 353ms/step - loss: 0.1276 - accuracy: 0.9519 - val_loss: 0.3890 - val_accuracy: 0.8761
Epoch 5/1024
299/299 [==============================] - ETA: 0s - loss: 0.1287 - accuracy: 0.9486
Epoch 00005: Reducing Max LR on Plateau: new max lr will be 1e-05 (if not early_stopping).
299/299 [==============================] - 106s 350ms/step - loss: 0.1287 - accuracy: 

In [14]:
learner.validate(class_names=distillbert_transformer.get_classes())

24/24 [==============================] - 12s 456ms/step
              precision    recall  f1-score   support

           0       0.96      0.92      0.94       699
           1       0.42      0.57      0.49        68

    accuracy                           0.89       767
   macro avg       0.69      0.75      0.71       767
weighted avg       0.91      0.89      0.90       767



array([[646,  53],
       [ 29,  39]])

In [15]:
distillbert_predictor = ktrain.get_predictor(learner.model, preproc=distillbert_transformer)

distillbert_test_data = test_df['sentence'].tolist()
distillbert_test_label = test_df['label'].tolist()

y_pred_distillbert = distillbert_predictor.predict(distillbert_test_data)

y_pred_distillbert = [int(x) for x in y_pred_distillbert]

tn, fp, fn, tp = confusion_matrix(distillbert_test_label, y_pred_distillbert).ravel()
print('True Negative: {}, False Positive: {}, False Negative: {}, True Positive: {}'.format(tn, fp, fn, tp))

print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

True Negative: 536, False Positive: 47, False Negative: 21, True Positive: 36
  Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.92      0.94       583
           1       0.43      0.63      0.51        57

    accuracy                           0.89       640
   macro avg       0.70      0.78      0.73       640
weighted avg       0.92      0.89      0.90       640
 



In [16]:
distillbert_predictor.save('../../../../saved_models/navigational_bert_base_cased_model_08012024')

: 

In [5]:
learner.validate(class_names=distillbert_transformer.get_classes())

51/51 [==============================] - 29s 427ms/step
              precision    recall  f1-score   support

    negative       0.98      0.95      0.97      1440
    positive       0.71      0.88      0.78       185

    accuracy                           0.94      1625
   macro avg       0.85      0.91      0.88      1625
weighted avg       0.95      0.94      0.95      1625



array([[1373,   67],
       [  23,  162]])